# Basic connectivity variables.
Only change them if values of lab are modified.

In [1]:
# Basic connectivity variables.
# Only change if values of lab are modified.
router = "192.168.10.4"
port = 830
username = "vagrant"
password = "vagrant"
protocol = "ssh"

In [2]:
from ydk.providers import CodecServiceProvider, NetconfServiceProvider
from ydk.services import CRUDService, CodecService
# We need to create a provider to connect with the router. 
# There is a considerable theory behind this step (netconf, restconfg, etc.), 
# but we'll leave the details aside for this practice.
provider = NetconfServiceProvider(address=router,
                                      port=port,
                                      username=username,
                                      password=password, 
                                      protocol=protocol)
crud = CRUDService()

# YDK "binds" data from/to the router into python objects. 
# Still, we might want to look at the responses in xml for a quick. The next objects allow us to do that.

cd_provider = CodecServiceProvider(type="xml")
codec = CodecService()

In [ ]:
# In case you want verbose logging, uncomment this lines.
#import logging
#log = logging.getLogger('ydk')
#log.setLevel(logging.DEBUG)
#ch = logging.StreamHandler()
#log.addHandler(ch)

# OSPF Configuration 
The module to use here will be Cisco-IOS-XR-ipv4-ospf-cfg.yang (https://github.com/YangModels/yang/blob/master/vendor/cisco/xr/611/Cisco-IOS-XR-ipv4-ospf-cfg.yang).

If you want to cheat, you can configure OSPF yourselves and see how the objects are created, or look the solution down.


In [4]:
from ydk.models.cisco_ios_xr import Cisco_IOS_XR_ipv4_ospf_cfg as xr_ipv4_ospf_cfg
# You will also need the next two.
from ydk.types import Empty
ospf = xr_ipv4_ospf_cfg.Ospf()  # create object

# read data from NETCONF device. This will be empty if you did not configure ISIS.
ospf = crud.read(provider, ospf)

Now,  try to configure ISIS on the router. If you want to test the configuration using the quagga router, please use a compatible configuration (i.e. Net and level type).

A potential solution for this excesise is down. Please note the the solution requires you to have configured the Loopback in the router.

In [ ]:
# Your solution goes here
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

The solution will only work if we configured the loopback in the previous exercise

In [8]:
ospf = xr_ipv4_ospf_cfg.Ospf() # create object
process = ospf.processes.Process()
process.process_name = "sdn"
# Note that the loopback should have 
process.default_vrf.router_id = "1.1.1.3"
process.start = Empty()

# Area 0
area_area_id = process.default_vrf.area_addresses.AreaAreaId()
area_area_id.area_id = 0
area_area_id.running = Empty()

# loopback interface passive
name_scope = area_area_id.name_scopes.NameScope()
name_scope.interface_name = "Loopback0"
name_scope.running = Empty()
name_scope.passive = True
area_area_id.name_scopes.name_scope.append(name_scope)

# gi0/0/0/0 interface
name_scope = area_area_id.name_scopes.NameScope()
name_scope.interface_name = "GigabitEthernet0/0/0/0"
name_scope.running = Empty()
# Note we'll do point to point, this should match with the configuration in quagga.
name_scope.network_type = xr_ipv4_ospf_cfg.OspfNetworkEnum.point_to_point
area_area_id.name_scopes.name_scope.append(name_scope)
process.default_vrf.area_addresses.area_area_id.append(area_area_id)


In [9]:
ospf_response = crud.create(provider, process)